<a href="https://githubtocolab.com/geonextgis/cropengine/blob/main/docs/examples/Run wofost (single location).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install the latest version of [cropengine](https://geonextgis.github.io/cropengine) if needed.

In [ ]:
# !pip install -U cropengine

## Import libraries

In [ ]:
import os
import pandas as pd
import seaborn as sns
from cropengine import WOFOSTCropSimulationRunner
from cropengine.agromanagement import WOFOSTAgroEventBuilder

## User inputs

## Instantiate crop simulation engine for WOFOST

In [ ]:
# Define the model name
MODEL_NAME = "Wofost72_WLP_CWB"

# Initialize Engine
runner = WOFOSTCropSimulationRunner(
    model_name=MODEL_NAME, workspace_dir="test_output/location_workspace"
)

In [ ]:
# Location
LATITUDE = 53.3721
LONGITUDE = 13.82299

# Crop Configuration
# Note: Use runner.get_..._options() to see valid values if unsure
models = runner.get_model_options()
crops = runner.get_crop_options(MODEL_NAME)
CROP_NAME = "sugarbeet"
varieties = runner.get_variety_options(MODEL_NAME, CROP_NAME)
CROP_VARIETY = "Sugarbeet_601"

# Timing
crop_start_end = runner.get_crop_start_end_options()
CAMPAIGN_START = "2006-01-01"
CROP_START = "2006-04-05"
CROP_START_TYPE = "emergence"
CROP_END_TYPE = "harvest"
CROP_END = "2006-10-20"
CAMPAIGN_END = "2007-01-01"
MAX_DURATION = 300

## Create agromanagements with user inputs

In [ ]:
agro_event_builder = WOFOSTAgroEventBuilder()

# Note: Use agro_event_builder.get_..._events_info() to see valid values if unsure
timed_events_info = agro_event_builder.get_timed_events_info()
state_events_info = agro_event_builder.get_state_events_info()

# Build timed events (irrigation)
irrigation_schedule = [
    {"event_date": "2006-05-25", "amount": 3.0, "efficiency": 0.7},
    {"event_date": "2006-06-30", "amount": 2.5, "efficiency": 0.7},
]

irrigation_events = agro_event_builder.create_timed_events(
    signal_type="irrigate", events_list=irrigation_schedule
)

# Build state Events (fertilization based on DVS)
nitrogen_schedule = [
    {"threshold": 0.3, "N_amount": 40, "N_recovery": 0.7},
    {"threshold": 0.6, "N_amount": 60, "N_recovery": 0.7},
    {"threshold": 1.12, "N_amount": 40, "N_recovery": 0.7},
]

nitrogen_events = agro_event_builder.create_state_events(
    signal_type="apply_n",
    state_var="DVS",
    zero_condition="rising",
    events_list=nitrogen_schedule,
)

## Prepare system (must be implemented before running the simulation)

In [ ]:
runner.prepare_system(
    latitude=LATITUDE,
    longitude=LONGITUDE,
    campaign_start=CAMPAIGN_START,
    campaign_end=CAMPAIGN_END,
    crop_start=CROP_START,
    crop_start_type=CROP_START_TYPE,
    crop_end_type=CROP_END_TYPE,
    crop_end=CROP_END,
    max_duration=MAX_DURATION,
    crop_name=CROP_NAME,
    variety_name=CROP_VARIETY,
    timed_events=[irrigation_events],
    state_events=[nitrogen_events],
    force_update=False,
    force_param_update=True,
    site_overrides={"WAV": 10},  # Extra site params can be passed as overrides
)

## Get the parameters table to be displayed in the dashboard

### Soil parameters

In [ ]:
soil_params = pd.read_csv(
    runner.files["soil_params"]
)  # To be displayed in the dashboard's 'soil parameters' table
print(soil_params.shape)
soil_params.head()

In [ ]:
site_params = pd.read_csv(
    runner.files["site_params"]
)  # To be displayed in the dashboard's 'site parameters' table
print(site_params.shape)
site_params.head()

In [ ]:
crop_params = pd.read_csv(
    runner.files["crop_params"]
)  # To be displayed in the dashboard's 'crop parameters' table
print(crop_params.shape)
crop_params.head()

## Run the Simulation

In [ ]:
simulation_output = runner.run_simulation(
    soil_overrides=None,  # Pass the user-edited soil parameters in a dictionary
    site_overrides=None,  # Pass the user-edited site parameters in a dictionary
    crop_overrides=None,  # Pass the user-edited crop parameters in a dictionary
    agro_file_path=None,  # Pass the user-uploaded agro_file_path if any.
)

# Set day as index
simulation_output = simulation_output.set_index("day")
print(simulation_output.shape)
simulation_output.head()

## Plot the simulation output

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

# Variables
cols = 2
n_vars = len(simulation_output.columns)
rows = math.ceil(n_vars / cols)

# Use a categorical color palette large enough
palette = sns.color_palette("tab20", n_vars)

fig, axes = plt.subplots(rows, cols, figsize=(14, 2 * rows), sharex=True)
axes = np.array(axes).reshape(-1)

for i, var in enumerate(simulation_output.columns):
    sns.lineplot(
        x=simulation_output.index,
        y=simulation_output[var],
        ax=axes[i],
        color=palette[i],
    )

# Hide unused axes
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
pd.DataFrame(runner.get_model_options())

- Wofost72_Phenology: Done
- Wofost72_PP: Done
- Wofost72_WLP_CWB: Done
- Wofost73_PP: Done
- Wofost73_WLP_CWB: Done